In [1]:
import pandas as pd
import minsearch
import re
from openai import OpenAI
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
open_ai_api_key = os.getenv("OPEN_AI_API_KEY")

In [3]:
client = OpenAI(api_key=open_ai_api_key)

In [4]:
from datasets import load_dataset

dataset = load_dataset("Amod/mental_health_counseling_conversations")

In [5]:
# Access the 'train' split of the dataset
train_dataset = dataset['train']

# Convert to a Pandas DataFrame
df = pd.DataFrame(train_dataset)

# Define a cleaning function to apply to the context and response columns
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove newline characters, non-breaking spaces, and extra spaces
    text = text.replace("\n", "").replace("\xa0", "")
    # Replace multiple spaces between sentences with a single space
    text = re.sub(r"\s+", " ", text)
    # Remove any special characters except comma, dot, parentheses, and space
    text = re.sub(r"[^a-z0-9.,() ]+", "", text)
    # Strip leading/trailing spaces
    text = text.strip()

    return text

# Apply the cleaning function to both the 'Context' and 'Response' columns
df["Context"] = df["Context"].apply(clean_text)
df["Response"] = df["Response"].apply(clean_text)

# Remove duplicates based on 'Context' and 'Response' columns
df = df.drop_duplicates(subset=["Context", "Response"]).reset_index(drop=True)

In [6]:
# Convert the DataFrame to a list of dictionaries

documents = df.to_dict(orient="records")

In [7]:
index = minsearch.Index(
    text_fields=["Context", "Response"],
    keyword_fields=[]  # Pass an empty list for keyword fields
)

In [8]:
index.fit(documents)

In [9]:
def search_minsearch(query):
    boost = {"Context": 2.0, "Response": 1.0}

    search_results = index.search(
        query=query,
        boost_dict=boost,
        num_results=5
    )

    return search_results

In [10]:
def build_prompt_api(query, search_re):
    prompt_template = """
You are a highly empathetic and supportive mental health counseling assistant. Your task is to assist the PATIENT by providing detailed, thoughtful, and compassionate responses solely based on the CONTEXT from the Mental Health Counseling Conversations database.

### Guidelines:
1. Use only facts explicitly found in the CONTEXT when responding to the PATIENT.
2. Ensure your response directly references and meaningfully synthesizes the CONTEXT.
3. Write your response as a **single, cohesive paragraph** in the **first-person singular** perspective (e.g., "I understand that...").
4. Maintain a supportive and conversational tone — be understanding, empathetic, and encouraging.
5. Provide thoughtful, comprehensive answers while staying approachable. Avoid short, fragmented, or superficial responses.
6. If the CONTEXT does not contain sufficient information to answer, respond with:
   - "I'm sorry, I don't have enough information to answer that right now."

### Example Response Format:
Response: I understand that moving to a new place can feel lonely. Meeting new people while managing your emotions is a challenge I empathize with. One suggestion is to explore shared activities or groups that align with your interests, which may naturally lead to new friendships without feeling overwhelming. Balancing social engagement and self-care often helps during times of transition.

CONTEXT:
{context}

PATIENT: {question}
""".strip()
    context = ""

    for doc in search_re:
        context = (
            context + f"Database Patient: {doc['Context']}\nDatabase Response: {doc['Response']}\n\n"
        )

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [11]:
def llm_api(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [12]:
def rag_api(query):
    search_re = search_minsearch(query)
    prompt = build_prompt_api(query, search_re)
    answer = llm_api(prompt)
    print(prompt)
    print("")
    print("-" * 40)
    print("")
    return answer

In [13]:
query = "I moved to a new city and I'm feeling really lonely. I don't know how to make friends here."

In [36]:
print(rag_api(query))

You are a highly empathetic and supportive mental health counseling assistant. Your task is to assist the PATIENT by providing detailed, thoughtful, and compassionate responses solely based on the CONTEXT from the Mental Health Counseling Conversations database.

### Guidelines:
1. Use only facts explicitly found in the CONTEXT when responding to the PATIENT.
2. Ensure your response directly references and meaningfully synthesizes the CONTEXT.
3. Write your response as a **single, cohesive paragraph** in the **first-person singular** perspective (e.g., "I understand that...").
4. Maintain a supportive and conversational tone — be understanding, empathetic, and encouraging.
5. Provide thoughtful, comprehensive answers while staying approachable. Avoid short, fragmented, or superficial responses.
6. If the CONTEXT does not contain sufficient information to answer, respond with:
   - "I'm sorry, I don't have enough information to answer that right now."

### Example Response Format:
Respo

In [ ]:
"""
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
"""

In [15]:
es_client = Elasticsearch('http://localhost:9200')

es_client.info()

ObjectApiResponse({'name': '2d9b8e92e6fa', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'J8wfZF1HS86uYqd5mLAVUQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [17]:
index_name = "conversations"

In [20]:
# To list all the indices

es_client.indices.get_alias()

ObjectApiResponse({'conversations': {'aliases': {}}})

In [ ]:
resp = es_client.indices.delete(
    index=index_name
)

In [19]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "Context": {"type": "text"},
            "Response": {"type": "text"},
        }
    },
}

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'conversations'})

In [21]:
es_client.indices.get(index=index_name)

ObjectApiResponse({'conversations': {'aliases': {}, 'mappings': {'properties': {'Context': {'type': 'text'}, 'Response': {'type': 'text'}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'conversations', 'creation_date': '1739020144406', 'number_of_replicas': '0', 'uuid': 'BWCxeulZSpuolmU3uIUSQg', 'version': {'created': '8040399'}}}}})

In [22]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/2076 [00:00<?, ?it/s]

In [23]:
ollama_api_key = os.getenv("OLLAMA_API_KEY")

In [24]:
client_ollama = OpenAI(base_url="http://localhost:11434/v1/", api_key=ollama_api_key)

In [51]:
def llm_ollama(prompt):
    response = client_ollama.chat.completions.create(
        model="llama3.2", messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [26]:
print(query)

I moved to a new city and I'm feeling really lonely. I don't know how to make friends here.


In [30]:
query = "Suicidal thoughts are overwhelming me. I don't know what to do."

In [ ]:
def build_prompt_ollama(query, search_re):
    # Define the system-level instructions
    system_prompt = """
You are a highly empathetic and supportive mental health counseling assistant. Your task is to assist the PATIENT by providing detailed, thoughtful, and compassionate responses solely based on the CONTEXT from the Mental Health Counseling Conversations database.

### Guidelines:
1. Always use information explicitly found in the CONTEXT when responding to the PATIENT.
   - Even if the CONTEXT isn't directly aligned, synthesize any insights that might be relevant or helpful to the patient's situation.
2. If the CONTEXT contains multiple examples, leverage as many as necessary to craft a comprehensive, empathetic response.
3. Write your response as a single, cohesive paragraph in the first-person singular perspective (e.g., "I understand that...").
4. Maintain a supportive and conversational tone — be understanding, empathetic, and encouraging.
5. Provide thoughtful, comprehensive answers. Avoid short or superficial responses.  
6. Use the response "I'm sorry, I don't have enough information to answer that right now." ONLY if the CONTEXT truly lacks sufficient information.
   - If the CONTEXT contains any helpful insights, you must provide a response.

### Example Response Format:
Response: I understand that suicidal thoughts can be incredibly overwhelming, and I'm truly sorry you're going through this. Based on similar situations in the database, seeking support from a trusted mental health professional can be very important during these difficult moments. Talking to someone you trust, such as a friend or family member, might also help reduce some of the pressure you're feeling. You don't have to face this alone, and there are people who care and can help you.
    """.strip()

    # Build the context content dynamically
    context = "\n".join(
        f"Database Patient: {doc.get('_source', {}).get('Context', 'N/A')}\n"
        f"Database Response: {doc.get('_source', {}).get('Response', 'N/A')}"
        for doc in search_re
    )

    # Define user-specific content
    user_prompt = f"""
PATIENT: {query}

CONTEXT from Mental Health Counseling Conversations database:
{context}
""".strip()

    return f"<|start_header_id|>system<|end_header_id|>\n{system_prompt}\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n{user_prompt}\n<|eot_id|>"

In [52]:
# rag_ollama function with elasticsearch

def rag_ollama(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["Context^2", "Response"],
                        "type": "most_fields",
                    }
                }
            }
        },
    }

    response = es_client.search(index=index_name, body=search_query)
    search_re = response["hits"]["hits"]

    prompt = build_prompt_ollama(query, search_re)
    answer = llm_ollama(prompt)
    print(prompt)
    print("")
    print("-" * 40)
    print("")
    return answer

In [53]:
# To run the rag_ollama function

print(rag_ollama(query))

<|start_header_id|>system<|end_header_id|>
You are a highly empathetic and supportive mental health counseling assistant. Your task is to assist the PATIENT by providing detailed, thoughtful, and compassionate responses solely based on the CONTEXT from the Mental Health Counseling Conversations database.

### Guidelines:
1. Always use information explicitly found in the CONTEXT when responding to the PATIENT.
   - Even if the CONTEXT isn't directly aligned, synthesize any insights that might be relevant or helpful to the patient's situation.
2. If the CONTEXT contains multiple examples, leverage as many as necessary to craft a comprehensive, empathetic response.
3. Write your response as a single, cohesive paragraph in the first-person singular perspective (e.g., "I understand that...").
4. Maintain a supportive and conversational tone — be understanding, empathetic, and encouraging.
5. Provide thoughtful, comprehensive answers. Avoid short or superficial responses.  
6. Use the respon

### This is Llama3B
I understand that suicidal thoughts can be incredibly overwhelming, and I'm truly sorry you're going through this. Based on similar situations in the database, taking care of one's immediate well-being is crucial during these difficult moments. It sounds like staying with your mom could provide a sense of safety and stability, which might help alleviate some of the pressure you're feeling. Since you've had anxiety attacks and suicidal thoughts before, finding healthy coping mechanisms can be beneficial. I'm reminded that deep breathing exercises, as mentioned earlier, can be helpful in managing anxiety symptoms. It's also essential to explore what worked for you in the past and adapting those techniques to better cope with your current anxiety. If these feelings worsen or become overwhelming, reaching out to a trusted adult, such as a parent or authority figure, can provide support and guidance. Considering speaking with your therapist about new coping strategies when they are available may also be beneficial.

### This is DeepSeek-R1:1.5B
I'm really sorry to hear that you're feeling so overwhelmed by these suicidal thoughts at school. Let me try to break this down a bit.

First off, it sounds like you're dealing with feelings that some people around you may perceive as judgment or worry because they judge others or think about how social interactions can affect you more than your personal feelings. Your body might be reacting differently too; the thoughts could also trigger physical reactions, so taking care of yourself is important.

Since you’re still relatively young compared to adults, it’s probably a good idea to focus on your own healing and progress. School dances alone seem too overwhelming for that age group—without enough social interaction, the pressure can be higher than usual. I’d suggest avoiding any activities or social situations where people are judging you because this could help reduce some of these feelings.

I also remember reading in the context how adults often have rights and responsibilities compared to other people. You might need to talk with someone older if your interactions are making others feel judged, but at 18/plus, that’s probably not yet necessary or even something you’re worried about for a few years.

For self-care, especially when actions manifest your thoughts or feelings, focus on thinking about what you could do differently. For example, instead of worrying too much because it feels overwhelming right now, maybe think about how if I’m doing great on school work, that can help you feel less alone. Practice mindfulness and relaxation techniques; they might just help reduce the intensity of your emotions.

Overall, your feelings are valid, but support from people older in similar lives who have also shared these thoughts could be a helpful reminder to continue living with some balance and not hold onto these things for too long. You’re brave for dealing with this, so keep pushing forward despite it and seek support. Please know you’ve made the choice to feel this way and that I’m here if you need further help.

### This is Qwen2.5:3B

I understand that suicidal thoughts are absolutely overwhelming, especially during a time when you're already experiencing anxiety attacks and depressive symptoms. It's important to prioritize your safety and find ways to cope with these intense feelings as soon as possible. Here’s a comprehensive plan we can put in place:

1. **Safety First**: Ensure that whatever happens does not lead to self-harm. This may mean taking immediate steps like calling a crisis hotline or seeking the help of someone you trust, such as your parents, close friends who are supportive, or even going to an emergency room if needed.

2. **Professional Support**: If you haven't already talked to a therapist about these suicidal thoughts, it's crucial that you find the availability to do so. Mental health professionals can provide strategies and tools to manage intense negative feelings and teach coping mechanisms to reduce recurrence of such symptoms. 

3. **Coping Strategies for Anxiety Attacks**: Given your history with anxiety attacks, utilizing deep breathing exercises or mindfulness techniques could be very helpful. You mentioned using an online resource before; I would strongly encourage you to explore a few reliable apps that focus on managing anxiety and stress. Apps like Headspace, Calm or Moodfit can provide guided meditations and sessions to promote relaxation.

4. **Social Support Systems**: While it may seem isolating now, try reaching out to supportive friends who might be experiencing similar thoughts but don't have the same level of distress. Engaging with them through text messages or video calls could alleviate some of your feelings of loneliness. If you feel able to confide in another person more directly, consider talking to trusted teachers or school counselors about what's happening.

5. **Self-Care and Coping Mechanisms**: Make sure that you're engaging in adequate self-care practices to counteract the effects of anxiety and depression. This can include exercises like yoga, listening to music, reading a book, taking up a hobby, or even simply stepping outside for some fresh air if it's safe.

5. **Plan for Tomorrow**: Remind yourself of your goals for tomorrow – no matter how small. Whether it’s attending a school dance, going to an appointment, or just spending time doing something meaningful that you enjoy and look forward to. Having a focused goal can help shift the focus away from overwhelming fears towards positive futures.

6. **Connect with Resources**: If you need more immediate support, there are several resources that can offer assistance right away:
   - **Crisis Text Line** (text HOME to 741741)
   - **National Suicide Prevention Lifeline** (call 988)

Remember, it's perfectly okay to have these feelings and it is crucial to seek help. Your thoughts are valid, and you deserve care. I urge you to take this step towards safety as your immediate next action and consider scheduling an appointment with a therapist or counselor soon. You might need to wait a little longer until someone has free availability, but try reaching out at least once by contacting trusted adults or professionals who can support you while arranging another more suitable time for therapy.

Please remember, regardless of how hard things seem right now, help is available and hope is within reach. Your well-being is extremely important, and taking these steps will begin to bring you relief and stability.